In [1]:
import re

import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('conll2000')
nltk.download('averaged_perceptron_tagger')

from transformers import BertConfig, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, \
                         AutoTokenizer, AutoModelForTokenClassification, pipeline, BertTokenizer, BertModel, \
                         LukeTokenizer, LukeForEntitySpanClassification

from tqdm import tqdm, trange

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import torch

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eloisedoyard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eloisedoyard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eloisedoyard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/eloisedoyard/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eloisedoyard/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
import os
import os.path

import re
import json
import pickle
import unicodedata
from functools import reduce
from itertools import groupby
from operator import itemgetter

import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from nameparser import HumanName
from nameparser.config import Constants

import fuzzy
import string
import editdistance
from libindic.soundex import Soundex
from stop_words import get_stop_words
#from python_parser import NameDenormalizer
from pyjarowinkler.distance import get_jaro_distance

import spacy
import torch

import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
#from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec, FastText
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertConfig, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, \
                         AutoTokenizer, AutoModelForTokenClassification, pipeline, BertTokenizer, BertModel, \
                         LukeTokenizer, LukeForEntitySpanClassification
BertTokenizer


transformers.models.bert.tokenization_bert.BertTokenizer

In [5]:
from transformers import CamembertModel, CamembertTokenizer
from transformers import FlaubertTokenizer, FlaubertModel, FlaubertForTokenClassification
from transformers import CamembertTokenizer, CamembertForTokenClassification

In [3]:
def get_book_text(gutenberg_id):
    '''Given a book ID, returns the book's text, excluding Project Gutenberg's header and outro.
    
     Parameters
    ----------
    gutenberg_id : int
        The book's Project Gutenberg ID

    Returns
    -------
    book_text : str
        The book's text, excluding Project Gutenberg's header and outro
    '''
    
    context = ''
    with open(f'../data/book/PG-{gutenberg_id}.txt', mode='r', encoding='utf-8') as f:
        context = f.read()
    return ' '.join([l for l in (context.split('End of the Project Gutenberg EBook of ')[0]
                                        .split('*** END OF THE PROJECT GUTENBERG EBOOK')[0]
                                        .split('\n')) if l][16:])

In [4]:
french_stopwords = pd.read_csv('../data/stopwords-fr.txt', header = None)[0].values.tolist()

In [16]:
def get_line_entities(l, ner_entities_tokens, ner_entities_words, sentence_index, tokenizer, nlp,
                     grouped_entities):
    '''Given a line, lists for tokens and words, and word index at the end of the sentence, as well as
    the tokenizer and nlp model instances (from huggingface's transformers), updates the tokens and
    words lists and the word index to include the given line.

    Parameters
    ----------
    l : str
        The line to analyze
    ner_entities_tokens : list
        A list containing all the Person tokens found so far, across all the previous lines
    ner_entities_words : list
        A list containing dictionary entries of all the Person entities found so far (the full 
        word corresponding to them (i.e. not separated tokens), their index in the sentence and 
        in the book overall, and their PER-entity classification score, a number between 0.0 and 
        1.0), across all the previous lines
    sentence_index : int
        The overall (book-wise) index of the first word of the sentence
    tokenizer : AutoTokenizer
        huggingface's tokenizer being used in the NER pipeline
    nlp : pipeline
        huggingface's NER pipeline object
    grouped_entities : bool
        Flag indicating whether the NER pipeline is configured to output grouped_entities or not

    Returns
    -------
    ner_entities_tokens : list
        A list containing all the Person tokens found so far, across this and all the previous lines
    ner_entities_words : list
        A list containing dictionary entries of all the Person entities found so far (the full 
        word corresponding to them (i.e. not separated tokens), their index in the sentence and 
        in the book overall, and their PER-entity classification score, a number between 0.0 and 
        1.0), across this and all the previous lines
    sentence_index : int
        The overall (book-wise) index of the first word of the next sentence
    '''
    new_entity_tokens = []
    if grouped_entities:
        new_entity_tokens = [e for e in nlp(l) if 'PER' in e['entity_group']]
    else:
        new_entity_tokens = [e for e in nlp(l) if 'PER' in e['entity']]
    ner_entities_tokens += new_entity_tokens
    # print('new entity tokens')
    # print(new_entity_tokens)
    tokenized_line = tokenizer(l)
    line_words = [w if w != 'word_tokenize_splits_cannot_into_2_words' else 'cannot'
                    for w in word_tokenize(
                                           re.sub(r'[^a-zA-Z0-9À-ÿ]', ' \g<0> ', 
                                                  l).replace('cannot', 
                                                            'word_tokenize_splits_cannot_into_2_words'))]
    # print('line words')
    # print(line_words)
    # go from token to word with
    for et in new_entity_tokens:
        if grouped_entities:
            # find index of grouped entity
            reconstructed_line = ' '.join([lw.lower() for lw in line_words])
            first_word = word_tokenize(re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' \g<0> ', et['word']))[0]
            if et['word'][0] == '#':
                first_word = word_tokenize(re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' \g<0> ', et['word'][2:]))[0]
            
            word_index = len(reconstructed_line[:reconstructed_line.index(first_word)].split())

            if et['word'] not in french_stopwords and et['word'].isalpha():
                # record grouped entity
                ner_entities_words += [{'full_word': et['word'], 
                                        'sentence_word_index': word_index, 
                                        'total_word_index': sentence_index+word_index,
                                        'score': et['score']}]
        else:
            # record non-grouped entity
            word_index = tokenized_line.word_ids()[et['index']]
            if line_words[word_index] not in french_stopwords and line_words[word_index].isalpha():
                ner_entities_words += [{'full_word': line_words[word_index], 
                                        'sentence_word_index': word_index, 
                                        'total_word_index': sentence_index+word_index,
                                        'score': et['score']}]
    sentence_index += len(line_words)
    return ner_entities_tokens, ner_entities_words, sentence_index

In [10]:
nlp(l)

[{'entity': 'I-LOC',
  'score': 0.55669767,
  'index': 10,
  'word': '▁d',
  'start': 36,
  'end': 38},
 {'entity': 'I-LOC',
  'score': 0.95578015,
  'index': 12,
  'word': 'Italie',
  'start': 39,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9981791,
  'index': 16,
  'word': '▁V',
  'start': 55,
  'end': 57},
 {'entity': 'I-LOC',
  'score': 0.9979278,
  'index': 17,
  'word': 'errière',
  'start': 57,
  'end': 64},
 {'entity': 'I-LOC',
  'score': 0.99829894,
  'index': 18,
  'word': 's',
  'start': 64,
  'end': 65},
 {'entity': 'I-PER',
  'score': 0.9855219,
  'index': 31,
  'word': '▁M',
  'start': 112,
  'end': 114},
 {'entity': 'I-PER',
  'score': 0.9845025,
  'index': 32,
  'word': '.',
  'start': 114,
  'end': 115},
 {'entity': 'I-PER',
  'score': 0.8523049,
  'index': 33,
  'word': '▁le',
  'start': 115,
  'end': 118},
 {'entity': 'I-PER',
  'score': 0.87298524,
  'index': 34,
  'word': '▁maire',
  'start': 118,
  'end': 124},
 {'entity': 'I-PER',
  'score': 0.92345726,
  'in

In [13]:
book_text =  get_book_text('798-8')
ner_model = 'mrm8488/mobilebert-finetuned-ner'
tokenizer = AutoTokenizer.from_pretrained(ner_model)
model = AutoModelForTokenClassification.from_pretrained(ner_model)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
sentence_level_book = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', book_text)

In [14]:
l=sentence_level_book[65]
a,b,c = get_line_entities(l, [], [], 0, tokenizer, nlp,
                     False)

new entity tokens
[{'entity': 'I-PER', 'score': 0.9980604, 'index': 42, 'word': 'm', 'start': 113, 'end': 114}, {'entity': 'I-PER', 'score': 0.9974752, 'index': 44, 'word': 'le', 'start': 116, 'end': 118}, {'entity': 'I-PER', 'score': 0.93767214, 'index': 45, 'word': 'mai', 'start': 119, 'end': 122}, {'entity': 'I-PER', 'score': 0.906227, 'index': 46, 'word': '##re', 'start': 122, 'end': 124}, {'entity': 'I-PER', 'score': 0.9871251, 'index': 48, 'word': 'jacob', 'start': 126, 'end': 131}, {'entity': 'I-PER', 'score': 0.990449, 'index': 49, 'word': '##in', 'start': 131, 'end': 133}, {'entity': 'I-PER', 'score': 0.955765, 'index': 51, 'word': 'bon', 'start': 137, 'end': 140}, {'entity': 'I-PER', 'score': 0.8932861, 'index': 54, 'word': '##ste', 'start': 146, 'end': 149}, {'entity': 'I-PER', 'score': 0.9953166, 'index': 97, 'word': 'm', 'start': 262, 'end': 263}, {'entity': 'I-PER', 'score': 0.9966055, 'index': 99, 'word': 'de', 'start': 265, 'end': 267}, {'entity': 'I-PER', 'score': 0.99

In [60]:
nlp(l)

[{'word': '▁d',
  'score': 0.5566989183425903,
  'entity': 'I-LOC',
  'index': 10,
  'start': 36,
  'end': 38},
 {'word': 'Italie',
  'score': 0.9557802677154541,
  'entity': 'I-LOC',
  'index': 12,
  'start': 39,
  'end': 45},
 {'word': '▁V',
  'score': 0.998179018497467,
  'entity': 'I-LOC',
  'index': 16,
  'start': 55,
  'end': 57},
 {'word': 'errière',
  'score': 0.9979278445243835,
  'entity': 'I-LOC',
  'index': 17,
  'start': 57,
  'end': 64},
 {'word': 's',
  'score': 0.9982990026473999,
  'entity': 'I-LOC',
  'index': 18,
  'start': 64,
  'end': 65},
 {'word': '▁M',
  'score': 0.9855217337608337,
  'entity': 'I-PER',
  'index': 31,
  'start': 112,
  'end': 114},
 {'word': '.',
  'score': 0.9845024347305298,
  'entity': 'I-PER',
  'index': 32,
  'start': 114,
  'end': 115},
 {'word': '▁le',
  'score': 0.8523029685020447,
  'entity': 'I-PER',
  'index': 33,
  'start': 115,
  'end': 118},
 {'word': '▁maire',
  'score': 0.8729842901229858,
  'entity': 'I-PER',
  'index': 34,
  's

In [6]:
def get_person_entities(gutenberg_id, grouped_entities=False, max_chunk_len=512, split_chunk_len=256):
    '''Given a book ID, returns its text (excluding Project Gutenberg's intro and outro), all its 
    tokens classified as PER (Person) entities, and all the words corresponding to those tokens, as 
    well as their index in the sentence and in the book, and their classification score as a PER entity.
    
    Parameters
    ----------
    gutenberg_id : int
        The book's Project Gutenberg ID
    grouped_entities : bool, optional
        Flag indicating whether the NER pipeline is configured to outout grouped_entities or not 
        (default is False)
    max_chunk_len : int, optional
        Maximum character-level length of each sentence passed to the model (default is 512)
    split_chunk_len : int, optional
        Maximum character-level length of each sub-sentence passed to the model, when splitting an
        overly big sentence into smaller sub-sentences (default is 256)

    Returns
    -------
    book_text : str
        The book's text, excluding Project Gutenberg's header and outro
    ner_entities_tokens : list
        A list containing all the Person tokens found across the whole book
    ner_entities_words : list
        A list containing dictionary entries of all the Person entities found across the whole book 
        (the full word corresponding to them (i.e. not separated tokens), their index in the sentence 
        and in the book overall, and their PER-entity classification score, a number between 0.0 and 
        1.0)
    '''
    # code is correct, but gives a warning about the model not having a predefined maximum length,
    # suppressing those warnings to not interfere with tdqm progress bar
    import warnings
    from transformers import logging
    warnings.filterwarnings('ignore')
    warnings.simplefilter('ignore')
    logging.set_verbosity_error()
    
    # read in gutenberg book
    book_text =  get_book_text(gutenberg_id)

    # load NER model and tokenizer
    ner_model = 'mrm8488/mobilebert-finetuned-ner'
    tokenizer = AutoTokenizer.from_pretrained(ner_model)
    model = AutoModelForTokenClassification.from_pretrained(ner_model)
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    
    # prepare for iteration over the book
    sentence_level_book = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', book_text)
    ner_entities_tokens = []
    ner_entities_words = []
    sentence_index = 0
    
    # iterate over sentence-level chunks
    for l in tqdm(sentence_level_book):
        l = l.lower()
        if len(l) > max_chunk_len:
            for m in range(len(l) // split_chunk_len + 1):
                new_l = ' '.join(l.split(' ')[m*split_chunk_len:][:(m+1)*split_chunk_len])
                ner_entities_tokens, ner_entities_words, sentence_index = get_line_entities(new_l, 
                                                                                            ner_entities_tokens, 
                                                                                            ner_entities_words,
                                                                                            sentence_index, 
                                                                                            tokenizer,
                                                                                            nlp,
                                                                                            grouped_entities)
        else:
            ner_entities_tokens, ner_entities_words, sentence_index = get_line_entities(l, 
                                                                                        ner_entities_tokens, 
                                                                                        ner_entities_words,
                                                                                        sentence_index, 
                                                                                        tokenizer, 
                                                                                        nlp,
                                                                                        grouped_entities)

    return book_text, ner_entities_tokens, ner_entities_words

In [17]:
rouge_noir_id = '798-8'
(rouge_noir_text, 
 rouge_noir_ent_tokens, 
 rouge_noir_ent_words) = get_person_entities(rouge_noir_id, grouped_entities=False)

100%|██████████| 7929/7929 [09:56<00:00, 13.28it/s]


In [25]:
# save entities into a dataframe, and to the disk
rouge_noir_df = pd.DataFrame(rouge_noir_ent_words)
# view top 25 entities
(rouge_noir_df
 .drop_duplicates('total_word_index')
 .groupby('full_word')
 .count()
 .sort_values(by='score', ascending=False)
)['score'][:25]

full_word
julien        1166
rênal          595
mathilde       340
mole           170
pirard         106
abbé            81
croisenois      53
sorel           53
derville        50
norbert         49
fervaques       38
élisa           29
valenod         28
napoléon        27
altamira        27
caylus          27
louis           24
trouva          23
chélan          21
amanda          19
geronimo        17
frilair         16
stanislas       16
maslon          14
don             14
Name: score, dtype: int64

In [21]:
# save entities into a dataframe, and to the disk
rouge_noir_df = pd.DataFrame(rouge_noir_ent_words)
# rouge_noir_df['full_word'] =  rouge_noir_df['full_word'].apply(lambda s: s.lower())
rouge_noir_df.to_csv('../data/book_dfs/rouge_noir_df.csv', index=False) 

In [51]:
# view top 25 entities
(rouge_noir_df
 .drop_duplicates('total_word_index')
 .groupby('full_word')
 .count()
 .sort_values(by='score', ascending=False)
)['score'][:50]

full_word
julien        450
é             369
mme           208
mathilde       98
è              75
valenod        70
mlle           63
mole           54
ê              48
marquis        43
ch             32
tait           31
sorel          29
re             24
fouqu          24
disait         21
tre            19
comte          19
pr             18
ç              17
grand          16
norbert        16
derville       15
û              14
croisenois     14
pensa          14
pirard         14
res            13
chevalier      11
nal            11
homme          11
monsieur       11
jour           10
mar            10
chambre        10
napol           9
saint           9
abb             9
tanbeau         8
dieu            8
esprit          8
appert          8
maire           7
ami             7
marquise        7
duc             7
amour           7
caract          7
altamira        7
voir            7
Name: score, dtype: int64

In [32]:
# save entities into a dataframe, and to the disk
rouge_noir_df = pd.DataFrame(rouge_noir_ent_words)
# rouge_noir_df['full_word'] =  rouge_noir_df['full_word'].apply(lambda s: s.lower())
rouge_noir_df.to_csv('../data/book_dfs/798-8.csv', index=False) 

# view top 25 entities
(rouge_noir_df
 .drop_duplicates('total_word_index')
 .groupby('full_word')
 .count()
 .sort_values(by='score', ascending=False)
)['score'][:25]

full_word
julien        1166
rênal          595
mathilde       340
mole           170
pirard         106
abbé            81
croisenois      53
sorel           53
derville        50
norbert         49
fervaques       38
élisa           29
valenod         28
napoléon        27
altamira        27
caylus          27
louis           24
trouva          23
chélan          21
amanda          19
geronimo        17
frilair         16
stanislas       16
maslon          14
don             14
Name: score, dtype: int64

In [20]:
len(stopwords.words('french'))

157

### Fine-tuned BERT embeddings

#### Fine-tune the (BERT) model

The following cells fine-tune the BERT Large Cased pre-trained model on a book. Indeed, the idea behind the fine-tuning is exactly to conduct it for the book which will later be analyzed using the later generated embeddings. To achieve this, a dummy task is used -- for this training, the task was identifying whether, given a random sentence of the book, it contained at least one (NER-identified) character in it.

The fine-tuning below is conducted for Charles Dicken's *David Copperfield* book.

In [34]:
def get_book_df(gutenberg_id, grouped_entities=False):
    '''Creates an Entities dataframe for the given book. If it already exists, just loads it from memory.

    Parameters
    ----------
    gutenberg_id : int
        The book's Project Gutenberg ID
    grouped_entities : bool, optional
        Flag indicating whether the NER pipeline used to create the entities was configured to output 
        grouped_entities or not (default is False)

    Returns
    -------
    book_df : DataFrame
        A DataFrame containing the book's entities, their sentence-wise and book-wise indexes and their
        PER-classification scores
    '''
    
    # check if df already exists on the disk
    book_csv_path = f'data/book_dfs/{gutenberg_id}.csv'
    if grouped_entities:
        book_csv_path = f'data/book_dfs/{gutenberg_id}_grouped.csv'
    if os.path.isfile(book_csv_path):
        return pd.read_csv(book_csv_path)
    
    # if df doesn't already exist, create it
    (book_text, book_ent_tokens, book_ent_words) = get_person_entities(gutenberg_id, 
                                                                       grouped_entities=grouped_entities)
    book_df = pd.DataFrame(book_ent_words)
    book_df['full_word'] =  book_df['full_word'].apply(lambda s: s.lower())
    
    # save df to disk and then return it
    book_df.to_csv(book_csv_path, index=False) 
    return book_df


In [35]:
book_df = get_book_df('798-8')
book_text = get_book_text('798-8')
sentence_level_book = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', book_text)

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentence_level_book]
labels = [1 if any([n in s for n in book_df['full_word'].unique()]) else 0 for s in sentences]


tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids=[]
for i in tqdm(range(len(tokenized_texts))):
    input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))

MAX_LEN = 512
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

100%|██████████| 7929/7929 [09:56<00:00, 13.28it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/book_dfs/798-8.csv'

In [ ]:
# generate the attention masks
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# creating the training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=56, test_size=0.2)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)


batch_size = 1

# create an iterator of both our training and validation data with torch DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = BertConfig.from_pretrained('bert-large-cased', output_hidden_states=True, num_labels=2)
model = BertForSequenceClassification.from_pretrained('bert-large-cased', config=config)
model.to(device) 

lr = 2e-5
max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1

# to reproduce BertAdam specific behavior set correct_bias=False
optimizer = AdamW(model.parameters(), lr=lr)  
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)  

In [ ]:
total_step = len(train_dataloader)
train_loss_set = []
epochs = 2

for epoch in tqdm(range(epochs)):
    model.train()

    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        train_loss_set.append(loss.item())    
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if (i) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

In [ ]:
torch.save(model, 'model_finetuning/BERT/798-8/checkpoints/finetuned_bert-large-cased.pt')